In [1]:
!pip install opencv-python
!pip install numpy

!pip install ffmpeg-python
!pip install tqdm

!pip install matplotlib

!pip install vidgear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.5 MB/s eta 0:00:00:00:0100:01


In [17]:
# FFmpeg Video Stabilization Implementation
# not that goodd

import os
import subprocess
import tempfile
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

def stabilize_video_ffmpeg(input_path, output_path, shakiness=5, accuracy=15, stepsize=6, smoothing=10):
    """
    Stabilize video using FFmpeg's vidstab filter.
    
    Args:
        input_path (str): Path to input unstable video
        output_path (str): Path to save stabilized video
        shakiness (int): How shaky is the video (1-10, default 5)
        accuracy (int): Accuracy of motion detection (1-15, default 15)
        stepsize (int): Step size of the search process (1-32, default 6)
        smoothing (int): How smooth the motion should be (1-100, default 10)
    
    Returns:
        bool: True if stabilization was successful, False otherwise
    """
    print(f"Stabilizing {input_path} to {output_path}")
    
    # Check if input file exists
    if not os.path.exists(input_path):
        print(f"Error: Input file does not exist: {input_path}")
        return False
    
    # Create a temporary file for the transform data
    with tempfile.NamedTemporaryFile(suffix='.trf', delete=False) as tmp_file:
        transform_file = tmp_file.name
    
    try:
        # First pass: analyze video and generate transform data
        analyze_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'vidstabdetect=shakiness={shakiness}:accuracy={accuracy}:stepsize={stepsize}:mincontrast=0.3:result={transform_file}',
            '-f', 'null', '-'
        ]
        
        print("Step 1: Analyzing video motion...")
        subprocess.run(analyze_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Second pass: apply stabilization using the transform data
        stabilize_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'vidstabtransform=input={transform_file}:zoom=0:smoothing={smoothing}:optalgo=gauss:maxshift=-1:interpol=bilinear',
            '-c:v', 'libx264',  # Use x264 codec
            '-preset', 'medium',  # Compression preset (adjust as needed)
            '-tune', 'film',  # Tune for film content
            '-crf', '18',  # Quality (lower is better, 18-28 is typical range)
            '-c:a', 'copy',  # Copy audio stream
            output_path
        ]
        
        print("Step 2: Applying stabilization...")
        subprocess.run(stabilize_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        print(f"Stabilization completed. Output saved to {output_path}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"Error during FFmpeg processing: {e}")
        print(f"Error output: {e.stderr.decode() if e.stderr else 'None'}")
        return False
    finally:
        # Clean up the temporary transform file
        if os.path.exists(transform_file):
            os.remove(transform_file)

# Function to test different parameters
def test_ffmpeg_parameters(input_path, output_dir, parameter_sets):
    """
    Test different parameter sets for FFmpeg stabilization
    
    Args:
        input_path (str): Path to input unstable video
        output_dir (str): Directory to save output videos
        parameter_sets (list): List of parameter dictionaries
    
    Returns:
        list: Paths to output videos
    """
    output_paths = []
    
    for i, params in enumerate(parameter_sets):
        output_name = f"stable_ffmpeg_test_{i+1}.mp4"
        output_path = os.path.join(output_dir, output_name)
        
        print(f"\nTest {i+1}: Parameters: {params}")
        success = stabilize_video_ffmpeg(input_path, output_path, **params)
        
        if success:
            output_paths.append(output_path)
    
    return output_paths

# Example usage
input_file = "../data/NissanMurano/unstable.mp4"
output_file = "../data/NissanMurano/stable_ffmpeg_claude.mp4"

# Make sure the output directory exists
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Stabilize with default parameters
stabilize_video_ffmpeg(input_file, output_file)

# Function to compare visual results
def compare_multiple_videos(video_paths, labels, num_frames=5):
    """
    Compare multiple videos side by side
    
    Args:
        video_paths (list): List of paths to videos to compare
        labels (list): Labels for each video
        num_frames (int): Number of frames to compare
    """
    # Open all videos
    caps = [cv2.VideoCapture(path) for path in video_paths]
    
    # Check if all videos opened successfully
    if not all(cap.isOpened() for cap in caps):
        print("Error: Could not open one or more videos")
        for cap in caps:
            cap.release()
        return
    
    # Get minimum frame count across all videos
    total_frames = min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) for cap in caps)
    
    # Sample frames evenly
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=int)
    
    # Set up the plot
    n_videos = len(video_paths)
    fig, axes = plt.subplots(num_frames, n_videos, figsize=(4*n_videos, 3*num_frames))
    
    # If there's only one frame to display, make sure axes is properly shaped
    if num_frames == 1:
        axes = axes.reshape(1, -1)
    
    for i, frame_idx in enumerate(frame_indices):
        for j, cap in enumerate(caps):
            # Set frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            
            # Read frame
            ret, frame = cap.read()
            
            if ret:
                # Convert BGR to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Display
                axes[i, j].imshow(frame_rgb)
                axes[i, j].set_title(f"{labels[j]}\nFrame {frame_idx}")
                axes[i, j].axis('off')
            else:
                axes[i, j].text(0.5, 0.5, "Frame not available", 
                               horizontalalignment='center',
                               verticalalignment='center')
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Release all video captures
    for cap in caps:
        cap.release()

# Uncomment to test different parameter sets
"""
parameter_sets = [
    {"shakiness": 3, "accuracy": 10, "stepsize": 6, "smoothing": 5},
    {"shakiness": 5, "accuracy": 15, "stepsize": 6, "smoothing": 10},
    {"shakiness": 8, "accuracy": 15, "stepsize": 4, "smoothing": 15}
]

output_dir = os.path.dirname(output_file)
output_paths = test_ffmpeg_parameters(input_file, output_dir, parameter_sets)

# Compare results
compare_multiple_videos(
    [input_file] + output_paths,
    ["Original"] + [f"Test {i+1}" for i in range(len(parameter_sets))],
    num_frames=3
)
"""

# Function to extract frames at specific angles
def extract_frames_at_angles(input_path, output_dir, num_frames=36):
    """
    Extract frames from a video at specific angles assuming a full 360° rotation
    
    Args:
        input_path (str): Path to input video
        output_dir (str): Directory to save extracted frames
        num_frames (int): Number of frames to extract (default 36 for 10° intervals)
    
    Returns:
        list: Paths to extracted frames
    """
    print(f"Extracting {num_frames} frames from {input_path}")
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open the video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_path}")
        return []
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate frame indices to extract (evenly distributed)
    frame_indices = np.linspace(0, total_frames-1, num_frames, dtype=int)
    
    extracted_paths = []
    
    for i, frame_idx in enumerate(frame_indices):
        # Set frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        
        # Read frame
        ret, frame = cap.read()
        
        if ret:
            # Calculate angle (0° to 350° for 36 frames)
            angle = (i * 360) // num_frames
            
            # Save frame
            output_path = os.path.join(output_dir, f"img_{i+1:02d}_{angle:03d}deg.png")
            cv2.imwrite(output_path, frame)
            
            extracted_paths.append(output_path)
            print(f"Extracted frame {i+1}/{num_frames} - {angle}° angle")
    
    cap.release()
    return extracted_paths

# Uncomment to extract frames after stabilization
"""
processed_dir = os.path.join("data/NissanMurano/processed")
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

extract_frames_at_angles(output_file, processed_dir)
"""

Stabilizing ../data/NissanMurano/unstable.mp4 to ../data/NissanMurano/stable_ffmpeg_claude.mp4
Step 1: Analyzing video motion...
Step 2: Applying stabilization...
Stabilization completed. Output saved to ../data/NissanMurano/stable_ffmpeg_claude.mp4


'\nprocessed_dir = os.path.join("data/NissanMurano/processed")\nif not os.path.exists(processed_dir):\n    os.makedirs(processed_dir)\n\nextract_frames_at_angles(output_file, processed_dir)\n'

In [20]:
#
# compare a few configs for best quality
#
#
#

import os
import subprocess
import tempfile
import time

def stabilize_video_ffmpeg(input_path, output_path, config_name, shakiness=8, accuracy=15, stepsize=4, smoothing=15, 
                          zoom=1.0, maxshift=-1, optalgo='gauss', interpol='bicubic', tripod=0, crf=15):
    """
    Stabilize video using FFmpeg's vidstab filter with configurable parameters.
    
    Args:
        input_path (str): Path to input unstable video
        output_path (str): Path to save stabilized video
        config_name (str): Name of this configuration for logging
        shakiness (int): How shaky is the video (1-10)
        accuracy (int): Accuracy of motion detection (1-15)
        stepsize (int): Step size of the search process (1-32)
        smoothing (int): How smooth the motion should be (1-100)
        zoom (float): Zooming factor to avoid black borders
        maxshift (int): Maximum number of pixels to shift frames
        optalgo (str): Algorithm for optimization (gauss or avg)
        interpol (str): Interpolation method (linear, bilinear, bicubic)
        tripod (int): Enable virtual tripod mode (0 or 1)
        crf (int): Constant Rate Factor for quality (lower is better)
    
    Returns:
        bool: True if stabilization was successful, False otherwise
    """
    print(f"Stabilizing with config: {config_name}")
    print(f"Parameters: shakiness={shakiness}, accuracy={accuracy}, stepsize={stepsize}, smoothing={smoothing}, "
          f"zoom={zoom}, maxshift={maxshift}, optalgo={optalgo}, interpol={interpol}, tripod={tripod}, crf={crf}")
    
    # Check if input file exists
    if not os.path.exists(input_path):
        print(f"Error: Input file does not exist: {input_path}")
        return False
    
    # Create a temporary file for the transform data
    with tempfile.NamedTemporaryFile(suffix='.trf', delete=False) as tmp_file:
        transform_file = tmp_file.name
    
    try:
        # First pass: analyze video and generate transform data
        analyze_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'vidstabdetect=shakiness={shakiness}:accuracy={accuracy}:stepsize={stepsize}:mincontrast=0.3:result={transform_file}',
            '-f', 'null', '-'
        ]
        
        print(f"Step 1: Analyzing video motion for {config_name}...")
        subprocess.run(analyze_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Second pass: apply stabilization using the transform data
        transform_params = f'vidstabtransform=input={transform_file}:zoom={zoom}:smoothing={smoothing}:optalgo={optalgo}:maxshift={maxshift}:interpol={interpol}'
        
        # Add tripod mode if enabled
        if tripod == 1:
            transform_params += ':tripod=1'
        
        stabilize_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', transform_params,
            '-c:v', 'libx264',  # Use x264 codec
            '-preset', 'slow',  # Slower preset for better quality
            '-tune', 'film',    # Tune for film content
            '-crf', str(crf),   # Quality setting
            '-c:a', 'copy',     # Copy audio stream
            output_path
        ]
        
        print(f"Step 2: Applying stabilization for {config_name}...")
        subprocess.run(stabilize_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        print(f"Stabilization completed for {config_name}. Output saved to {output_path}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"Error during FFmpeg processing for {config_name}: {e}")
        print(f"Error output: {e.stderr.decode() if e.stderr else 'None'}")
        return False
    finally:
        # Clean up the temporary transform file
        if os.path.exists(transform_file):
            os.remove(transform_file)

def create_test_batch(input_file, base_output_dir):
    """
    Create multiple versions of stabilized video with different configurations
    
    Args:
        input_file (str): Path to the unstable input video
        base_output_dir (str): Directory to save the output videos
    """
    # Create the test directory if it doesn't exist
    test_dir = os.path.join(base_output_dir, "stabilization_tests")
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        print(f"Created directory: {test_dir}")
    
    # Configuration sets for testing
    configs = [
        # Config 1: Original with increased smoothing
        {
            "name": "moderate_smooth",
            "params": {
                "shakiness": 8, 
                "accuracy": 15, 
                "stepsize": 4, 
                "smoothing": 30,  # Increased from 15
                "zoom": 1.05,    # Slight zoom to avoid borders
                "maxshift": -1, 
                "optalgo": "gauss", 
                "interpol": "bicubic",
                "tripod": 0,
                "crf": 15
            }
        },
        # Config 2: Higher smoothing with slight tripod effect
        {
            "name": "higher_smooth",
            "params": {
                "shakiness": 9, 
                "accuracy": 15, 
                "stepsize": 4, 
                "smoothing": 40,  # Higher smoothing
                "zoom": 1.1,     # More zoom to avoid black borders
                "maxshift": -1, 
                "optalgo": "gauss", 
                "interpol": "bicubic",
                "tripod": 0,
                "crf": 15
            }
        },
        # Config 3: Tripod mode with moderate smoothing
        {
            "name": "tripod_mode",
            "params": {
                "shakiness": 9, 
                "accuracy": 15, 
                "stepsize": 4, 
                "smoothing": 30,
                "zoom": 1.15,    # More zoom to compensate for tripod mode
                "maxshift": -1, 
                "optalgo": "gauss", 
                "interpol": "bicubic",
                "tripod": 1,     # Enable tripod mode
                "crf": 15
            }
        },
        # Config 4: Max smoothing with quality focus
        {
            "name": "max_smooth_quality",
            "params": {
                "shakiness": 10, 
                "accuracy": 15, 
                "stepsize": 4, 
                "smoothing": 50,  # Very high smoothing
                "zoom": 1.12,     # Higher zoom
                "maxshift": -1, 
                "optalgo": "gauss", 
                "interpol": "bicubic",
                "tripod": 0,
                "crf": 15        # Maintain high quality
            }
        },
        # Config 5: Balanced approach with tripod influence
        {
            "name": "balanced_tripod",
            "params": {
                "shakiness": 10, 
                "accuracy": 15, 
                "stepsize": 6,   # Increased for more precision
                "smoothing": 40, 
                "zoom": 1.15,
                "maxshift": 40,  # Limited max shift
                "optalgo": "gauss", 
                "interpol": "bicubic",
                "tripod": 1,     # Enable tripod mode
                "crf": 15
            }
        },
        # Config 6: Maximum stabilization with quality balance
        {
            "name": "max_stable_balanced",
            "params": {
                "shakiness": 10, 
                "accuracy": 15, 
                "stepsize": 6, 
                "smoothing": 60,  # Very high smoothing
                "zoom": 1.2,      # Higher zoom to avoid borders
                "maxshift": 30,   # Limited max shift
                "optalgo": "gauss", 
                "interpol": "bicubic",
                "tripod": 1,      # Enable tripod mode
                "crf": 18         # Slightly lower quality for better performance
            }
        }
    ]
    
    # Process each configuration
    for config in configs:
        # Create output filename with configuration details
        output_name = f"stable_{config['name']}.mp4"
        output_path = os.path.join(test_dir, output_name)
        
        # Apply stabilization with this configuration
        start_time = time.time()
        success = stabilize_video_ffmpeg(
            input_path=input_file,
            output_path=output_path,
            config_name=config['name'],
            **config['params']
        )
        elapsed_time = time.time() - start_time
        
        if success:
            print(f"Completed {config['name']} in {elapsed_time:.2f} seconds")
        else:
            print(f"Failed to process {config['name']}")
        
        print("-" * 50)

# Main execution
if __name__ == "__main__":
    # Extract the directory of the unstable video
    input_file = "../data/NissanMurano/unstable.mp4"
    base_dir = os.path.dirname(input_file)
    
    # Create test batch
    create_test_batch(input_file, base_dir)
    
    print("\nAll test configurations completed. Check the 'stabilization_tests' directory for results.")
    print("Configurations tested:")
    print("1. moderate_smooth - Original with increased smoothing")
    print("2. higher_smooth - Higher smoothing with slight zoom")
    print("3. tripod_mode - Tripod mode with moderate smoothing")
    print("4. max_smooth_quality - Maximum smoothing with quality focus")
    print("5. balanced_tripod - Balanced approach with tripod influence")
    print("6. max_stable_balanced - Maximum stabilization with quality balance")

Created directory: ../data/NissanMurano/stabilization_tests
Stabilizing with config: moderate_smooth
Parameters: shakiness=8, accuracy=15, stepsize=4, smoothing=30, zoom=1.05, maxshift=-1, optalgo=gauss, interpol=bicubic, tripod=0, crf=15
Step 1: Analyzing video motion for moderate_smooth...
Step 2: Applying stabilization for moderate_smooth...
Stabilization completed for moderate_smooth. Output saved to ../data/NissanMurano/stabilization_tests/stable_moderate_smooth.mp4
Completed moderate_smooth in 93.97 seconds
--------------------------------------------------
Stabilizing with config: higher_smooth
Parameters: shakiness=9, accuracy=15, stepsize=4, smoothing=40, zoom=1.1, maxshift=-1, optalgo=gauss, interpol=bicubic, tripod=0, crf=15
Step 1: Analyzing video motion for higher_smooth...
Step 2: Applying stabilization for higher_smooth...
Stabilization completed for higher_smooth. Output saved to ../data/NissanMurano/stabilization_tests/stable_higher_smooth.mp4
Completed higher_smooth i

In [5]:
# ffmpeg-claude-max-smooth-quality
# FFmpeg Video Stabilization Implementation - Max Smooth Quality
# works best - although not perfect

import os
import subprocess
import tempfile

def stabilize_video_ffmpeg(input_path, output_path):
    """
    Stabilize video using FFmpeg's vidstab filter with max smooth quality configuration.
    
    Args:
        input_path (str): Path to input unstable video
        output_path (str): Path to save stabilized video
    
    Returns:
        bool: True if stabilization was successful, False otherwise
    """
    print(f"Stabilizing {input_path} to {output_path}")
    
    # Check if input file exists
    if not os.path.exists(input_path):
        print(f"Error: Input file does not exist: {input_path}")
        return False
    
    # Create a temporary file for the transform data
    with tempfile.NamedTemporaryFile(suffix='.trf', delete=False) as tmp_file:
        transform_file = tmp_file.name
    
    try:
        # First pass: analyze video and generate transform data
        analyze_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'vidstabdetect=shakiness=10:accuracy=15:stepsize=4:mincontrast=0.3:result={transform_file}',
            '-f', 'null', '-'
        ]
        
        print("Step 1: Analyzing video motion...")
        subprocess.run(analyze_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Second pass: apply stabilization using the transform data
        stabilize_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'vidstabtransform=input={transform_file}:zoom=1.12:smoothing=50:optalgo=gauss:maxshift=-1:interpol=bicubic',
            '-c:v', 'libx264',  # Use x264 codec
            '-preset', 'slow',  # Slower preset for better quality
            '-tune', 'film',    # Tune for film content
            '-crf', '15',       # Higher quality (lower value)
            '-c:a', 'copy',     # Copy audio stream
            output_path
        ]
        
        print("Step 2: Applying stabilization...")
        subprocess.run(stabilize_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        print(f"Stabilization completed. Output saved to {output_path}")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"Error during FFmpeg processing: {e}")
        print(f"Error output: {e.stderr.decode() if e.stderr else 'None'}")
        return False
    finally:
        # Clean up the temporary transform file
        if os.path.exists(transform_file):
            os.remove(transform_file)

# Example usage
if __name__ == "__main__":
    input_file = "../data/NissanMurano/unstable.mp4"
    output_file = "../data/NissanMurano/stable_max_smoothing.mp4"
    
    # Make sure the output directory exists
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Stabilize with max smooth quality parameters
    stabilize_video_ffmpeg(input_file, output_file)

Stabilizing ../data/NissanMurano/unstable.mp4 to ../data/NissanMurano/stable_max_smoothing.mp4
Step 1: Analyzing video motion...
Step 2: Applying stabilization...
Stabilization completed. Output saved to ../data/NissanMurano/stable_max_smoothing.mp4


In [28]:
import os
import cv2
import numpy as np
import subprocess

def inject_exif(image_path, camera_make="Samsung", camera_model="Galaxy S21", focal_length="4.5", sensor_width="5.76"):
    """
    Injects smartphone-like EXIF metadata into an image for Meshroom.

    Args:
        image_path (str): Path to the image file.
        camera_make (str): Smartphone brand (e.g., Samsung, Apple)
        camera_model (str): Smartphone model (e.g., iPhone 13, Pixel 6)
        focal_length (str): Approximate focal length in mm.
        sensor_width (str): Approximate sensor width in mm.
    """
    cmd = [
        "exiftool",
        f"-Make={camera_make}",
        f"-Model={camera_model}",
        f"-FocalLength={focal_length}",
        f"-SensorWidth={sensor_width}",
        "-overwrite_original",
        image_path
    ]
    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def extract_frames_with_metadata(input_path, output_dir, num_frames=90):
    """
    Extract frames from a video at specific angles and inject metadata.

    Args:
        input_path (str): Path to input video
        output_dir (str): Directory to save extracted frames
        num_frames (int): Number of frames to extract

    Returns:
        list: Paths to extracted frames
    """
    print(f"Extracting {num_frames} frames from {input_path}")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    extracted_paths = []

    for i, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            angle = (i * 360) // num_frames
            output_path = os.path.join(output_dir, f"car{angle:03d}deg.jpg")
            cv2.imwrite(output_path, frame)

            # Inject EXIF metadata for Meshroom
            inject_exif(output_path)

            extracted_paths.append(output_path)
            print(f"Extracted and fixed metadata for frame {i+1}/{num_frames} - {angle}°")

    cap.release()
    return extracted_paths

# Example usage
if __name__ == "__main__":
    input_file = "../data/NissanMurano/stable.mp4"
    output_dir = "../data/NissanMurano/frames"

    extracted_frames = extract_frames_with_metadata(input_file, output_dir, num_frames=90)
    print(f"Extracted {len(extracted_frames)} frames with metadata for Meshroom.")


Extracting 90 frames from ../data/NissanMurano/stable.mp4
Extracted and fixed metadata for frame 1/90 - 0°
Extracted and fixed metadata for frame 2/90 - 4°
Extracted and fixed metadata for frame 3/90 - 8°
Extracted and fixed metadata for frame 4/90 - 12°
Extracted and fixed metadata for frame 5/90 - 16°
Extracted and fixed metadata for frame 6/90 - 20°
Extracted and fixed metadata for frame 7/90 - 24°
Extracted and fixed metadata for frame 8/90 - 28°
Extracted and fixed metadata for frame 9/90 - 32°
Extracted and fixed metadata for frame 10/90 - 36°
Extracted and fixed metadata for frame 11/90 - 40°
Extracted and fixed metadata for frame 12/90 - 44°
Extracted and fixed metadata for frame 13/90 - 48°
Extracted and fixed metadata for frame 14/90 - 52°
Extracted and fixed metadata for frame 15/90 - 56°
Extracted and fixed metadata for frame 16/90 - 60°
Extracted and fixed metadata for frame 17/90 - 64°
Extracted and fixed metadata for frame 18/90 - 68°
Extracted and fixed metadata for fra

In [4]:
#
# Uses the Vidgear library - is not that good.
#

import cv2
from vidgear.gears.stabilizer import Stabilizer

def stabilize_video_vidgear(input_path, output_path):
    """
    Stabilizes a shaky video using Vidgear's Stabilizer.

    Parameters:
        input_path (str): Path to the input shaky video.
        output_path (str): Path to save the stabilized video.
    """
    # Initialize Stabilizer with default parameters
    stabilizer = Stabilizer()

    # Open input video
    cap = cv2.VideoCapture(input_path)
    
    if not cap.isOpened():
        print("❌ Error: Could not open video file.")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize Video Writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Process each frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Stabilize frame
        stable_frame = stabilizer.stabilize(frame)

        # Write stabilized frame to output
        if stable_frame is not None:
            out.write(stable_frame)

    print(f"✅ Stabilized video saved at: {output_path}")

    # Release resources
    cap.release()
    out.release()
    stabilizer.clean()

# Run stabilization
input_video = "../data/NissanMurano/unstable.mp4"
output_video = "../data/NissanMurano/stable_vidgear.mp4"
stabilize_video_vidgear(input_video, output_video)


✅ Stabilized video saved at: ../data/NissanMurano/stable_vidgear.mp4
